In [207]:
#importation 
import os
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [209]:
#connexion a jumia
options = Options()
driver = webdriver.Firefox(options=options)


In [306]:
def scrapper_jumia(driver, nom_article_scraper):
    """
    Fonction pour scraper Jumia.
    Prend en paramètre le nom du produit et retourne un DataFrame contenant les données.
    """
    # Dictionnaire pour stocker les données scrappées
    data = {
        "Nom_Article": [],
        "Prix_Article": [],
        "Lien": [],
    }

    # URL de la page Jumia
    base_url = "https://www.jumia.ci/"
    driver.get(base_url)
    while driver.execute_script("return document.readyState") != "complete":
        pass

    # Localisation et interaction avec la barre de recherche
    try:
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "fi-q"))
        )
        search_bar.clear()
        search_bar.send_keys(nom_article_scraper)
        search_bar.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"Erreur lors de l'accès à la barre de recherche : {e}")
        return pd.DataFrame(data)

    # Boucle pour parcourir les pages
    while True:
        # Attendre que les articles soient chargés
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "prd._fb.col.c-prd"))
            )
        except Exception as e:
            print(f"Erreur lors du chargement des articles : {e}")
            break

        # Analyse de la page avec BeautifulSoup
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

        # Extraction des articles
        article_elements = soup.find_all("article", class_="prd _fb col c-prd")
        for article in article_elements:
            try:
                name = article.find("h3", class_="name").text.strip().lower()
                price_text = article.find("div", class_="prc").text
                price = float(price_text.replace("FCFA", "").replace(",", "").strip())
                link = base_url + article.find("a", href=True)["href"]
                data["Nom_Article"].append(name)
                data["Prix_Article"].append(price)
                data["Lien"].append(link)
            except Exception as e:
                continue

        # Pagination - Aller à la page suivante si disponible
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.pg[aria-label='Page suivante']")
            next_page_url = next_button.get_attribute("href")
            driver.get(next_page_url)
            time.sleep(2)  # Laisser du temps pour charger la page suivante
        except Exception:
            print("Fin de la pagination.")
            break

    # Retourner les données sous forme de DataFrame
    return pd.DataFrame(data)


In [308]:
def scrapper_kevajo(driver, nom_article_scraper):
    """
    Fonction pour scraper Kevajo.
    Prend en paramètre le driver Selenium et le nom du produit, et retourne un DataFrame contenant les données.
    """

    # Dictionnaire pour stocker les données scrappées
    data = {
        "Nom_Article": [],
        "Prix_Article": [],
        "Lien": [],
    }

    # URL de la page Kevajo
    base_url = "https://kevajo.com/"
    driver.get(base_url)

    # Assurer que la page est complètement chargée
    while driver.execute_script("return document.readyState") != "complete":
        pass

    try:
        # Rechercher la barre de recherche et entrer le produit
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Cherchez un produit"]'))
        )
        search_bar.clear()
        search_bar.send_keys(nom_article_scraper)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(5)  # Attendre le chargement des résultats
    except Exception as e:
        print(f"Erreur lors de l'accès à la barre de recherche : {e}")
        return pd.DataFrame(data)

    # Boucle pour parcourir les pages
    while True:
        try:
            # Attendre que les articles soient chargés
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-source='main_loop']"))
            )

            # Obtenir le code source de la page
            page = driver.page_source
            soup = BeautifulSoup(page, "html.parser")

            # Trouver tous les articles dans la section 'main_loop'
            article_elements = soup.find_all("div", {"data-loop": True})
            for article in article_elements:
                try:
                    name = article.find("h3", {"class": "wd-entities-title"}).text.strip()
                    price_text = article.find("bdi").text
                    price = float(price_text.replace("CFA", "").replace(",", "").strip())
                    link = article.find("h3", {"class": "wd-entities-title"}).find("a", href=True)["href"]
                    
                    # Ajouter les données extraites
                    data["Nom_Article"].append(name)
                    data["Prix_Article"].append(price)
                    data["Lien"].append(link)
                except AttributeError as e:
                    continue

        except Exception as e:
            print(f"Erreur lors du chargement des articles : {e}")
            break

        # Pagination - Aller à la page suivante si disponible
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
            next_page_url = next_button.get_attribute("href")
            driver.get(next_page_url)
            time.sleep(2)  # Laisser du temps pour charger la page suivante
        except Exception:
            print("Fin de la pagination ou bouton 'Next' non trouvé.")
            break

    return pd.DataFrame(data)

In [310]:
def scrapper_djokstore(driver, nom_article_scraper):
    """
    Fonction pour scraper Kevajo.
    Prend en paramètre le driver Selenium et le nom du produit, et retourne un DataFrame contenant les données.
    """

    # Dictionnaire pour stocker les données scrappées
    data = {
        "Nom_Article": [],
        "Prix_Article": [],
        "Lien": [],
    }

    # URL de la page Kevajo
    base_url = "https://djokstore.ci/"
    driver.get(base_url)

    # Assurer que la page est complètement chargée
    while driver.execute_script("return document.readyState") != "complete":
        pass

    try:
        # Rechercher la barre de recherche et entrer le produit
        search_bar = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.header-search .search_header input.search_header__input:not([type=submit]):not([type=checkbox])'))
        )
        search_bar.clear()
        search_bar.send_keys(nom_article_scraper)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(5)  # Attendre le chargement des résultats
    except Exception as e:
        print(f"Erreur lors de l'accès à la barre de recherche : {e}")
        return pd.DataFrame(data)

    # Boucle pour parcourir les pages
    while True:
        try:
            # Attendre que les articles soient chargés
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#product-grid"))
            )

            # Obtenir le code source de la page
            page = driver.page_source
            soup = BeautifulSoup(page, "html.parser")

            # Trouver tous les articles dans la section 'main_loop'
            article_elements = soup.select(".product-collection")
            for article in article_elements:
                try:
                    name = article.select_one("h2").text.strip()
                    price_text = article.select_one(".current").text
                    price = float(price_text.replace("FCFA", "").replace(",", "").strip())
                    link = "https"+article.select_one(".d-block.image_product.cursor-default.ratio")["href"]
                    
                    # Ajouter les données extraites
                    data["Nom_Article"].append(name)
                    data["Prix_Article"].append(price)
                    data["Lien"].append(link)
                except AttributeError as e:
                    continue
        except Exception as e:
            print(f"Erreur lors du chargement des articles : {e}")
            break

        # Pagination - Aller à la page suivante si disponible
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, ".page-next.pagination__item.pagination__item--prev.pagination__item-arrow.link.motion-reduce")
            next_page_url = next_button.get_attribute("href")
            driver.get(next_page_url)
            time.sleep(2)  # Laisser du temps pour charger la page suivante
        except Exception:
            print("Fin de la pagination ou bouton 'Next' non trouvé.")
            break

    return pd.DataFrame(data)

In [312]:
def scraper_les_produits(liste_produis,site):
    # DataFrame pour stocker toutes les données
    resultats_totaux = pd.DataFrame()
    # Boucle pour scrapper les données de chaque marque
    for marque in marques:
        print(f"Scraping les articles pour la marque : {marque}")
        try:
            data_marque = pd.DataFrame()
            # Appeler la fonction scrapper_jumia
            if site == "jumia":
                data_marque = scrapper_jumia(driver, marque)
                # Ajouter une colonne pour indiquer la marque
            elif site == "kevajo" :
                data_marque = scrapper_kevajo(driver, marque)
            elif site == "djokstore":
                data_marque = scrapper_djokstore(driver, marque)
            data_marque["Marque"] = marque
            # Ajouter les résultats au DataFrame total
            resultats_totaux = pd.concat([resultats_totaux, data_marque], ignore_index=True)
        except Exception as e:
            print(f"Erreur lors du scraping pour {marque} : {e}")
    return resultats_totaux

In [314]:
marques = ["Samsung","iphone","Xiaomi","huawei","tecno"]

In [316]:
jumia_data = scraper_les_produits(marques,'jumia')
jumia_data.to_csv("resultats_scrapping_jumia.csv", index=False)

Scraping les articles pour la marque : Samsung
Fin de la pagination.
Scraping les articles pour la marque : iphone
Fin de la pagination.
Scraping les articles pour la marque : Xiaomi
Fin de la pagination.
Scraping les articles pour la marque : huawei
Fin de la pagination.
Scraping les articles pour la marque : tecno
Fin de la pagination.


In [317]:
kevajo_data = scraper_les_produits(marques,'kevajo')
kevajo_data.to_csv("resultats_scrapping_kevajo.csv", index=False)


Scraping les articles pour la marque : Samsung
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : iphone
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : Xiaomi
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : huawei
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : tecno
Fin de la pagination ou bouton 'Next' non trouvé.


In [318]:
djokstore_data = scraper_les_produits(marques,'djokstore')
djokstore_data.to_csv("resultats_scrapping_djokstore.csv", index=False)

Scraping les articles pour la marque : Samsung
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : iphone
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : Xiaomi
Erreur lors du chargement des articles : 'NoneType' object is not subscriptable
Scraping les articles pour la marque : huawei
Fin de la pagination ou bouton 'Next' non trouvé.
Scraping les articles pour la marque : tecno
Fin de la pagination ou bouton 'Next' non trouvé.


Marque
Samsung    93
iphone     70
tecno      43
huawei     14
Xiaomi     10
Name: count, dtype: int64

In [324]:
kevajo_data

,Nom_Article,Prix_Article,Lien,Marque
0,Powerbank solaire 40000Mah d’origine rapide po...,25000.0,https://kevajo.com/boutique/telephones-et-tabl...,Samsung
1,Chargeur adaptateur type C 45W compatible Sams...,6800.0,https://kevajo.com/boutique/telephones-et-tabl...,Samsung
2,Chargeur Samsung Super Rapide – Noir,9000.0,https://kevajo.com/boutique/telephones-et-tabl...,Samsung
3,Chargeur Samsung Charge Turbo – Noir,25000.0,https://kevajo.com/boutique/telephones-et-tabl...,Samsung
4,Télécommande Adapté Samsung Noir,12000.0,https://kevajo.com/boutique/tv-audio-video/tel...,Samsung
...,...,...,...,...
607,Smart Tecnology TV LED HD 32″ – Régulateur & D...,64000.0,https://kevajo.com/boutique/electronique/tv/tv...,tecno
608,Etui à Rabat Compatible Tecno Pova Neo – Noir,10000.0,https://kevajo.com/boutique/telephones-et-tabl...,tecno
609,Etui à Rabat Compatible Tecno Pova Neo – Bleu,10000.0,https://kevajo.com/boutique/telephones-et-tabl...,tecno
610,Chargeurs Rapide Très Puissant – Tecno,5000.0,https://kevajo.com/boutique/telephones-et-tabl...,tecno
